In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

2023-09-09 15:07:54.348329: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-09 15:07:54.384013: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-09 15:07:54.384536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 15:07:54.982744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("sabzi.csv")


In [3]:
# Exclude the 'Dates' column for training
vegetable_data = data.drop('Dates', axis=1)

In [4]:

# Prepare sequences and targets for all vegetables
X, y = [], []
scaler = MinMaxScaler()
sequence_length = 10
for i in range(len(vegetable_data)):
    vegetable_data_scaled = scaler.fit_transform(vegetable_data.iloc[i].values.reshape(-1, 1))
    
    for j in range(len(vegetable_data_scaled) - sequence_length):
        X.append(vegetable_data_scaled[j : j + sequence_length])
        y.append(vegetable_data_scaled[j + sequence_length])

X = np.array(X) 
y = np.array(y)



In [5]:
# Split data into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define a function to build the LSTM model with given hyperparameters
def build_model(num_units):
    model = Sequential([
        LSTM(units=num_units, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train the model on all vegetables
model = build_model(64)  # You can choose the best hyperparameters based on your evaluation

model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

2023-09-09 15:07:55.871204: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 15:07:55.872343: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
# Predict for the selected vegetable and date
selected_date = input("Enter the custom date (yyyy/mm/dd): ")
selected_vegetable = input("Enter the custom vegetable name: ")
# Prepare the input sequence for prediction
selected_data = data[['Dates', selected_vegetable]]
selected_data.set_index('Dates', inplace=True)
selected_sequence = selected_data.iloc[-sequence_length:]
selected_sequence_scaled = scaler.fit_transform(selected_sequence.values.reshape(-1, 1))

# Predict the price for the selected vegetable and date
predicted_price_scaled = model.predict(np.array([selected_sequence_scaled]))
predicted_price = scaler.inverse_transform(predicted_price_scaled)

print(f"Predicted {selected_vegetable} price on {selected_date}: {round(predicted_price[0][0])}")

Enter the custom date (yyyy/mm/dd):  2023/09/09
Enter the custom vegetable name:  Onion


1/1 [==============================] - 0s 118ms/step
Predicted Onion price on 2023/09/09: 30


In [7]:
# model.save("Predictor.h5")